In [1]:
# Import necessary modules
from finlab import data, backtest
import talib
import os

current_path= os.path.dirname(os.getcwd())
import finlab
finlab.login(open(f"{current_path}\config.txt", "r").read())

import warnings
# 忽略FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)


# Get financial data
當月營收 = data.get("monthly_revenue:當月營收")
revenue_growth_rate = data.get('fundamental_features:營收成長率')
roe = data.get('fundamental_features:ROE稅後')
pb_ratio = data.get('price_earning_ratio:股價淨值比')
volume = data.get('price:成交股數')
close = data.get('price:收盤價')

# 計算月營收斜率
def rev_growth_func(rev):
    return talib.LINEARREG_SLOPE(rev, timeperiod=12) / talib.LINEARREG_INTERCEPT(rev,12)

rev_grouwth = 當月營收.apply(rev_growth_func)

# Define selection criteria
revenue_growth_threshold = revenue_growth_rate.index_str_to_date().quantile(0.8, axis=1)
roe_threshold = roe.mean(axis=1)

# Find stocks meeting criteria
revenue_growth_stocks = revenue_growth_rate > revenue_growth_threshold
high_roe_stocks = (roe > roe.index_str_to_date().quantile(0.6, axis=1)).deadline()
rev_sl_compare = (當月營收.average(3) > 當月營收.average(12)) & (當月營收 > 當月營收.shift(12))

# Define buy condition
buy_condition = (

    # 把 ROE > 80% 改成 ROE > 50%，ROE 在台股會有回歸效應（ROE高不可持續，反而傷害股價）
    high_roe_stocks 

    # 增加月營收成長的部分（台股才有月營收，資料公布時間更快）
    & (當月營收.pct_change().rolling(12).std() < 0.3)
    & rev_sl_compare
    
    # 把本益比換成股價淨值比（本益比在台股上直接用比較困難）
    & (pb_ratio < 2.2)  # 2 

    # 增加流動性
    & (volume > 300000)

    # 回測後發現波動（風險）太大，選擇低風險特性的股票
    & (close.pct_change().rolling(60).std() < 0.025)  #原本是60 , 0.025
)

# 月營收增長最快的股票
p = rev_grouwth[buy_condition].is_largest(12)

# 調整換股時間為月營收截止日
p = p.reindex(當月營收.index_str_to_date().index)

# Simulate positions
result = backtest.sim(p, position_limit=0.2, resample='D', name='巴菲特選股法優化版')

<>:8: SyntaxWarning: invalid escape sequence '\c'
<>:8: SyntaxWarning: invalid escape sequence '\c'
C:\Users\danie\AppData\Local\Temp\ipykernel_21372\2278974615.py:8: SyntaxWarning: invalid escape sequence '\c'
  finlab.login(open(f"{current_path}\config.txt", "r").read())


輸入成功!
Daily usage: 26.9 / 5000 MB - etl:financial_statements_disclosure_dates
Daily usage: 29.0 / 5000 MB - etl:is_flagged_stock
